In [100]:
import time
start = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [101]:
import time
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.functions import explode
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col,row_number

# l = [10,18,20]
# df.filter(F.col("score").isin(l))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [102]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Reading all datasets needed from s3

### Reading User followings

In [103]:
file_path_uf = "s3://aws-glue-neptune-data/postgre-data-lake/user_followings_list_v2/partition=2022-02-03/"
user_followings_list = spark.read.format('parquet').load(file_path_uf).\
select('user_id', 'following_ids','following_count')
user_followings_list = user_followings_list.filter("following_count>0 AND following_count<=300").\
select('user_id', 'following_ids')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
user_followings_list.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- following_ids: array (nullable = true)
 |    |-- element: integer (containsNull = true)

### Reading User folllowers

In [105]:
file_path_ufollowers = "s3://aws-glue-neptune-data/postgre-data-lake/user_followers_list/"
user_followers_list = spark.read.format('parquet').load(file_path_ufollowers).\
select('user_id', 'follower_ids')
#user_followers_list = user_followers_list.filter("followers_count>0 AND followers_count<=300").\
#select('user_id', 'follower_ids')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
user_followers_list.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- follower_ids: array (nullable = true)
 |    |-- element: integer (containsNull = true)

### Reading Users table

In [107]:
file_path_users= "s3://aws-glue-neptune-data/postgre-data-lake/users/partition=2022-02-03/"
users = spark.read.format('parquet').load(file_path_users).\
select('id', 'current_language')
users= users.withColumnRenamed("id","user_id")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
users.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- current_language: short (nullable = true)

### People Score

In [109]:
file_path_scores= "s3://aws-glue-neptune-data/postgre-data-lake/user_scores_log_v2/partition=2022-02-03/"
people_score = spark.read.format('parquet').load(file_path_scores).\
select('user_id','language' ,'final_score')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
people_score.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- language: short (nullable = true)
 |-- final_score: decimal(10,2) (nullable = true)

### Consideration Set

In [111]:
file_path_users= "s3://ml-test-analytics/test/stg_rfy/consideration_set/cs_users_20220203_173818.csv"
cs_users_df = spark.read.csv(
    file_path_users, 
    header=True)
cs_users_df = cs_users_df.select('user_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
cs_users_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: string (nullable = true)

In [113]:
cs_users_df = cs_users_df.withColumnRenamed("user_id","cs_user_id")
cs_users_df = cs_users_df.withColumn("cs_user_id",cs_users_df["cs_user_id"].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### For testing with a smaller subset 

In [114]:
#cs_users_df = cs_users_df.limit(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Following list consideration set

In [115]:
cs_ufl = cs_users_df.join(user_followings_list, 
                               cs_users_df.cs_user_id == user_followings_list.user_id,
                               "inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Exploding Users following

In [116]:
cs_ufl_exp = cs_ufl.select(cs_ufl.cs_user_id,explode(cs_ufl.following_ids).alias('cs_following_user_id'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
cs_ufl_exp.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)
 |-- cs_following_user_id: integer (nullable = true)

### Followers list consideration set

In [118]:
cs_ufol = cs_users_df.join(user_followers_list, 
                               cs_users_df.cs_user_id == user_followers_list.user_id,
                               "inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Exploding Users followers

In [119]:
cs_ufol_exp = cs_ufol.select(cs_ufol.cs_user_id,explode(cs_ufol.follower_ids).alias('cs_follower_user_id'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
cs_ufol_exp.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)
 |-- cs_follower_user_id: integer (nullable = true)

#### Subtracting user's followers from the user's following, is what they can follow back

In [121]:
fb_potential_df =cs_ufol_exp.subtract(cs_ufl_exp)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
fb_potential_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)
 |-- cs_follower_user_id: integer (nullable = true)

In [123]:
# Now we have to filter folks whose language are same as cs_user_id

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
# Getting the user id language
fb_potential_df = fb_potential_df.join(users, 
                               fb_potential_df.cs_user_id == users.user_id,
                               "left").select(fb_potential_df['cs_user_id'],fb_potential_df['cs_follower_user_id'],users['current_language'].alias('cl_cs_user_id'))
# Getting the follower id language
fb_potential_df = fb_potential_df.join(users, 
                               fb_potential_df.cs_follower_user_id == users.user_id,
                               "left").select(fb_potential_df['cs_user_id'],fb_potential_df['cs_follower_user_id'],fb_potential_df['cl_cs_user_id'],users['current_language'].alias('cl_cs_follower_user_id'))

# Filter Recommendations where language is not matching
fb_potential_df= fb_potential_df.filter("cl_cs_user_id=cl_cs_follower_user_id")

#fb_potential_df= fb_potential_df.filter("cl_cs_user_id=cl_cs_follower_user_id").drop('cl_cs_user_id','cl_cs_follower_user_id')
#fb_potential_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
# Now filter folks whose people score are greater than 2 
# and also dropping follower users whose people score in same language is null or not found

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
fb_potential_df  = fb_potential_df .join(people_score,
                                        (fb_potential_df.cs_follower_user_id == people_score.user_id) &
                                        (fb_potential_df.cl_cs_follower_user_id == people_score.language),
                                        how='left').\
select(fb_potential_df['cs_user_id'],fb_potential_df['cs_follower_user_id'], people_score['final_score']).\
filter("final_score>=2.0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
#fb_potential_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
fb_potential_df.write.parquet("s3://ml-test-analytics/test/stg_rfy/stg_fb/dryrun/",mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
window = Window.partitionBy(fb_potential_df['cs_user_id']).orderBy(fb_potential_df['final_score'].desc())
fb_ranked= fb_potential_df.select('*', row_number().over(window).alias('rank')).\
  filter(col('rank') <= 100).select('cs_user_id', 'cs_follower_user_id','final_score')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
#fb_ranked.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
fb_ranked.write.parquet("s3://ml-test-analytics/test/stg_rfy/stg_fb/dryrun-top100/",mode="overwrite")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
end = time.time()
print(end - start)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

67.01146674156189